In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
def distance(stress_x, stress_y, crack_x, crack_y):
    return ((stress_x-crack_x)**2+(stress_y-crack_y)**2)**1/2

def stress(v, r):
    return (1-1/(1+np.exp(r-v)))*(r-v)

def probability(stress_x, stress_y, radius, cracks):
    sum = []
    for name, par in cracks.items():
        v = distance(stress_x, stress_y, par['crack_x'], par['crack_y'])
        s = stress(v, radius)
        e = -(s/par['sf'])**par['m']
        sum.append(e)

        print(f'v({name}) = {v:.3f}')
        print(f's({name}) = {s:.3f}')
        print(f'e({name}) = {e:.3f}')
        print('----')
    print(f'exp_sum = {np.sum(sum):.3f}')
    print('----')

    return 1-np.exp(np.sum(sum))

In [ ]:
def stress_gradient(stress_x, stress_y):
    inner_radius = 0.
    outer_radius = 1.2
    halo_color = 'red'
    center_color = 'black'  ## redish with 25% alpha
    xmin = stress_x - outer_radius
    xmax = stress_x + outer_radius
    ymin = stress_y - outer_radius
    ymax = stress_y + outer_radius
    x, y = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
    r = np.sqrt((x - stress_x) ** 2 + (y - stress_y) ** 2)
    z = np.where(r < inner_radius, np.nan, np.clip(outer_radius - r, 0, np.inf))
    cmap = LinearSegmentedColormap.from_list('', ['#FFFFFF00', halo_color])
    cmap.set_bad(center_color)

    extend=[xmin, xmax, ymin, ymax]

    return z, extend, cmap

def find_marker_coor(base, cumulative, value):
    cumulative = np.asarray(cumulative)
    idx = (np.abs(cumulative - value)).argmin()
    return base[idx], value

In [ ]:
data = np.random.randn(1000)

def plot_prob(stress_x, stress_y, i):
    fig, ax = plt.subplots(2,1, figsize=(5,10))

    circle1 = plt.Circle((0, 0), 1, color='k', fill=False, lw=3)
    ax[0].plot([-0.2,0.3], [0.2, 0.6], 'k', lw=3)
    ax[0].plot([-0.5,0.3], [-0.2, -0.6], 'k', lw=3)
    ax[0].plot([0.4,0.5], [-0.2, 0.6], 'k', lw=3)

    ax[0].set_aspect(1)
    ax[0].add_artist(circle1)
    ax[0].set_xlim(-2, 2)
    ax[0].set_ylim(-2, 2)
    ax[0].set_axis_off()

    z, extend, cmap = stress_gradient(stress_x, stress_y)
    ax[0].imshow(z, cmap=cmap, extent=extend, origin='lower', zorder=3)

    # some fake data

    # evaluate the histogram
    values, base = np.histogram(data, bins=40)
    #evaluate the cumulative
    cumulative = np.cumsum(values)/1000
    # plot the cumulative function
    ax[1].plot(base[:-1], cumulative, c='blue')

    marker_x, marker_y = find_marker_coor(base, cumulative, min(stress_x*stress_y, 1))
    ax[1].plot(marker_x, marker_y, 'ro', markersize=10)

    fig.savefig(f'plots/stress_gradient_{i:02d}.png', dpi=300, bbox_inches='tight')


In [ ]:
stress_x_arr = np.linspace(0,1,11)
stress_y_arr = np.linspace(0,1,11)

for i in range(len(stress_x_arr)):
    stress_x = stress_x_arr[i]
    stress_y = stress_y_arr[i]
    plot_prob(stress_x, stress_y, i)

In [ ]:
stress_x = 1.
stress_y = 1.
radius = 1

cracks = {
    'crack_1': {
            'crack_x': 0.5,
            'crack_y': 0.5,
            'sf': 0.3,
            'm': 3,
            },
    'crack_2': {
            'crack_x': -0.5,
            'crack_y': -0.5,
            'sf': 0.4,
            'm': 2,
            },
    'crack_3': {
        'crack_x': 0.5,
        'crack_y': -0.5,
        'sf': 0.3,
        'm': 2,
        },
    }

p = probability(stress_x, stress_y, radius, cracks)
print(f'p = {p:.3f}%')